In [1]:
%%capture
!pip install shapely
!pip install geopandas
!pip install folium
!pip install config
!pip install timezonefinder


In [2]:
#Function to get isolines  
from shapely.geometry import Polygon
from shapely import wkt
import requests
import json
import geopandas as gpd
from geopandas import GeoSeries
import folium
import config
import pandas as pd
import timezonefinder as tf
from timezonefinder import TimezoneFinder
tf = TimezoneFinder()
from datetime import datetime, timezone, date, time, timedelta


In [3]:
# Get LatLon from address
def get_latlon(event_address):
    get_latlon = f'https://geocoder.ls.hereapi.com/6.2/geocode.json?searchtext={event_address}&gen=9&apiKey=CvCsf7byXWXESq3FtlGpWXUWRqYEnj-W8k9amDXCJJ4'
    js_latlon = requests.get(url = get_latlon).json()['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']
    return js_latlon
# Get UTC Timezone offset based on LatLon
def get_offset(*, Latitude, Longitude):
    from datetime import datetime
    from pytz import timezone, utc
    today = datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lng=Longitude, lat=Latitude))
    today_target = tz_target.localize(today)
    today_utc = utc.localize(today)
    return(int((today_utc - today_target).total_seconds() / 3600))
# Format Arrival Time in Isoformat
def get_isotime(start_time, event_date, utc_offset):
    t_24 = datetime.strptime(start_time,'%I:%M%p')
    d = datetime.strptime(event_date,'%Y-%m-%d')
    dt_24 = datetime.combine(d.date(),t_24.timetz(),tzinfo=timezone(timedelta(hours=utc_offset)))
    arrival_time = dt_24.isoformat(sep='T')
    return arrival_time

# Get Isoline from HERE API
def get_isodata(lat,lon,
                trip_mode,
                trip_duration,
                arrival_time):   
    try: 
        obtain_isoline = (f'https://isoline.route.ls.hereapi.com/routing/7.2/calculateisoline.json' +
        f'?apiKey=CvCsf7byXWXESq3FtlGpWXUWRqYEnj-W8k9amDXCJJ4'+
        f'&mode=fastest;{trip_mode}'+
        f'&destination=geo!{lat},{lon}'+
        f'&range={trip_duration}'+
        f'&rangetype=time'+
        f'&resolution=10'+
        f'&singlecomponent=true'+
        f'&arrival={arrival_time}')
        js_isoline = requests.get(url = obtain_isoline).json()['response']['isoline'][0]['component'][0]['shape']
        coords = Polygon([(float(x.split(',')[1]), float(x.split(',')[0])) for x in js_isoline])
        geojs = gpd.GeoSeries([coords])
        geojs.crs = {'init' : 'epsg:4326'}
        return geojs
    except KeyError:
        js = requests.get(url).json()
        print(js)
        raise ValueError("HereAPI doesn't have data requested")
    except IndexError:
        print(js)
        raise ValueError("HereAPI doesn't have quality data")
        

In [11]:
# User input event details
EVENT_DICT = {'anamosa':['102 Chamber Dr Anamosa, IA 52205','11:00AM','2020-01-03'],
              'waterloo':['1712 W 4th St Waterloo, IA 50701','2:00PM','2020-01-03'],
              'decorah':['900 E Main St, Decorah, IA 52101','5:00PM','2020-01-03'],
              'dubuque':['1800 Clarke Dr Dubuque, IA 52001','11:00AM','2020-01-04'],
              'grundy':['705 F Ave Grundy Center, IA 50638','2:30PM','2020-01-04'],
              'mason':['308 S Pennsylvania Ave, Mason City, IA 50401','5:00PM','2020-01-04'],
              'boone':['1900 Lakewood Dr Boone, IA 50036','10:00AM','2020-01-05']
             }

# Output table
OUTPUT_TABLE = 'bernie_nmarchio2.iowa_event_isolines'

# Max and min travel range in seconds (and increment of isoline series)
max_range = 5400
min_range = 300
time_increment = 300 


In [5]:
event_object = []
event_gpd = gpd.GeoDataFrame()
for i in EVENT_DICT.items():
    event_coords = get_latlon(event_address=i[1][0].replace(' ','%20'))
    hour_offset = get_offset(**event_coords)
    timestamp = get_isotime(start_time=str(i[1][1]),
                            event_date=str(i[1][2]),
                            utc_offset=hour_offset)
    for commute_mode in ['car','pedestrian']:
        for commute_time in range(min_range, max_range, time_increment):
            if (commute_mode == 'pedestrian' and commute_time <= 1800) or commute_mode == 'car':
                    isoline = get_isodata(lat=event_coords['Latitude'],
                                          lon=event_coords['Longitude'],
                                          trip_mode=commute_mode,
                                          trip_duration=commute_time,
                                          arrival_time=timestamp)
                    df = pd.DataFrame({'event_input':[i],
                                       'event_address':[i[1][0]],
                                       'event_latlon':[event_coords],
                                       'event_time':[timestamp],
                                       'trip_mode': [commute_mode],
                                       'trip_duration_minutes': [commute_time/60]
                                      })
                    isoline_gdf = gpd.GeoDataFrame(df, geometry = isoline)
                    isoline_gdf.crs = {'init':'epsg:4326'}
                    event_gpd = pd.concat([event_gpd, isoline_gdf])


In [6]:
event_gpd.head(n=5)

,event_input,event_address,event_latlon,event_time,trip_mode,trip_duration_minutes,geometry
0,"(anamosa, [102 Chamber Dr Anamosa, IA 52205, 1...","102 Chamber Dr Anamosa, IA 52205","{'Latitude': 42.10527, 'Longitude': -91.26578}",2020-01-03T11:00:00-06:00,car,5.0,"POLYGON ((-91.31278 42.07146, -91.29639 42.071..."
0,"(anamosa, [102 Chamber Dr Anamosa, IA 52205, 1...","102 Chamber Dr Anamosa, IA 52205","{'Latitude': 42.10527, 'Longitude': -91.26578}",2020-01-03T11:00:00-06:00,car,10.0,"POLYGON ((-91.40591 42.06390, -91.40350 42.063..."
0,"(anamosa, [102 Chamber Dr Anamosa, IA 52205, 1...","102 Chamber Dr Anamosa, IA 52205","{'Latitude': 42.10527, 'Longitude': -91.26578}",2020-01-03T11:00:00-06:00,car,15.0,"POLYGON ((-91.50478 42.04742, -91.49689 42.047..."
0,"(anamosa, [102 Chamber Dr Anamosa, IA 52205, 1...","102 Chamber Dr Anamosa, IA 52205","{'Latitude': 42.10527, 'Longitude': -91.26578}",2020-01-03T11:00:00-06:00,car,20.0,"POLYGON ((-91.54976 42.04056, -91.54907 42.044..."
0,"(anamosa, [102 Chamber Dr Anamosa, IA 52205, 1...","102 Chamber Dr Anamosa, IA 52205","{'Latitude': 42.10527, 'Longitude': -91.26578}",2020-01-03T11:00:00-06:00,car,25.0,"POLYGON ((-91.61980 42.02820, -91.61636 42.026..."


In [9]:
#event_gpd.to_file("countries.geojson", driver='GeoJSON')


In [12]:
create_geojson_df = civis.io.dataframe_to_civis(df=event_gpd,
                                                database='Bernie 2020', 
                                                table= 'bernie_nmarchio2.isoline_geometries', 
                                                existing_table_rows='drop')
create_geojson_df.result()

{'created_at': '2019-12-31T17:26:59.000Z',
 'error': None,
 'finished_at': '2019-12-31T17:27:09.000Z',
 'id': 195964769,
 'started_at': '2019-12-31T17:26:59.000Z',
 'state': 'succeeded'}

In [13]:
convert_to_geometry_sql = f'''
DROP TABLE IF EXISTS {OUTPUT_TABLE};
CREATE TABLE {OUTPUT_TABLE}
  DISTKEY (event_input) AS (select 
          event_input
         ,event_address
         ,event_latlon
         ,event_time
         ,trip_mode
         ,trip_duration_minutes
         ,geometry as isoline
         ,ST_GeomFromText(geometry,4326) as geometry 
from (select * from bernie_nmarchio2.isoline_geometries))'''

output_table = civis.io.query_civis(sql=convert_to_geometry_sql, database='Bernie 2020')
output_table.result().state

'succeeded'